In [1]:
import os
import random
import pickle
import torch
import torchvision
import pandas as pd
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True

In [4]:
import torchvision.transforms as transforms
from torch import nn

In [7]:
class EmbeddingNet(nn.Module):
    """EmbeddingNet using ResNet-101."""

    def __init__(self, resnet):
        """Initialize EmbeddingNet model."""
        super(EmbeddingNet, self).__init__()

        # Everything except the last linear layer
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        num_ftrs = resnet.fc.in_features
        self.fc = nn.Linear(num_ftrs, 4096)

    def forward(self, x):
        """Forward pass of EmbeddingNet."""
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out


class TripletNet(torch.nn.Module):
    """Triplet Network."""

    def __init__(self, embeddingnet):
        """Triplet Network Builder."""
        super(TripletNet, self).__init__()
        self.embeddingnet = embeddingnet

    def forward(self, a, p, n):
        """Forward pass."""
        # anchor
        embedded_a = self.embeddingnet(a)

        # positive examples
        embedded_p = self.embeddingnet(p)

        # negative examples
        embedded_n = self.embeddingnet(n)

        return embedded_a, embedded_p, embedded_n
    
    
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

In [9]:
model = torch.load('../data/pickles/models/24.12.0.15.pth')

In [10]:
embd_net = model.embeddingnet

In [11]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [12]:
# Normalize training set together with augmentation
one_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
#     transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.0069], [0.0033])
])

train_dir = '../data/spectrograms/train/train/'
val_dir = '../data/spectrograms/train/val/'
test_dir = '../data/spectrograms/test/'

batch_size = 16
test_dataset = ImageFolderWithPaths(test_dir, one_transform)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=8
)

train_dataset = ImageFolderWithPaths(train_dir, one_transform)
val_dataset = ImageFolderWithPaths(val_dir, one_transform)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=8
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=8
)

# class_names = train_dataset.classes


# Loading Tiny ImageNet dataset
# trplt_path = '../pickles_for_git/triplets.p'
# batch_szie = 32
# trainset = TripletDataset(triplet_path=trplt_path, transform=one_transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_szie, num_workers=8)

# testset = TripletImageLoader(
#     base_path=root, triplets_filename="", transform=transform_test, train=False)
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=batch_size_test, num_workers=32)

In [13]:
len(test_dataloader), len(test_dataset)

(7, 97)

In [14]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs, x, path in test_loader:
            inputs = inputs.to(device)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = torch.nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [15]:
embd_net = embd_net.to(device)

In [16]:
with torch.no_grad():
    logits = []

    for inputs, x, path in test_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        break

In [17]:
logits[0][0].shape

torch.Size([4096])

In [18]:
with torch.no_grad():
    logits = []
    path_list = list()
    for inputs, x, path in test_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [19]:
with torch.no_grad():
#     logits = []
#     path_list = list()
    for inputs, x, path in val_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [20]:
with torch.no_grad():
#     logits = []
#     path_list = list()
    for inputs, x, path in train_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [21]:
path_list[0][0]

'../data/spectrograms/test/unknown/000190.png'

In [22]:
logits[0][0].shape[0]

4096

In [23]:
emb_list = list()
for i in logits:
    for tens in i:
        emb_list.append(tens.numpy())

In [24]:
embeddings = pd.DataFrame(emb_list)

In [25]:
track_list = list()
for i in path_list:
    for path in i:
        name = path.split('/')[-1]
        ids = name.split('.')[0]
        track_list.append(ids)

In [26]:
embeddings['id'] = track_list

In [27]:
embeddings = embeddings[['id'] + list(np.arange(logits[0][0].shape[0]))]

In [28]:
embeddings

,id,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,000190,-0.098584,0.061195,-0.419897,-0.547398,-0.370505,-0.346276,0.174820,0.120204,-0.275544,...,-0.360226,-0.083810,0.240440,-0.139375,0.587489,0.326977,0.266145,0.440282,-0.074168,0.535761
1,000194,-0.232637,-0.040279,-0.337808,-0.594535,-0.365719,-0.361804,0.183087,0.232225,-0.299450,...,-0.313092,-0.069227,0.208865,-0.142131,0.597543,0.365311,0.396611,0.430305,-0.005214,0.525710
2,000667,-0.379676,0.236807,-0.321998,-0.436458,-0.143315,-0.499157,-0.018364,0.091818,-0.748339,...,-0.113017,0.171769,0.271148,-0.422386,0.066315,0.520129,0.695829,0.654227,-0.366704,0.240918
3,001040,-0.359856,0.092744,-0.198099,-0.400031,-0.346778,-0.366024,-0.014941,-0.000961,-0.667051,...,-0.355462,0.061808,0.185608,-0.402459,0.372494,0.606056,0.570929,0.537257,-0.110737,0.278486
4,001686,-0.345710,-0.027553,-0.309630,-0.537911,-0.210654,-0.444954,0.008424,-0.033211,-0.359033,...,-0.201391,0.084446,0.155775,-0.232248,0.569569,0.308580,0.157916,0.549588,-0.088380,0.511589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7989,149416,-0.339247,0.275376,-0.295336,-0.381422,-0.261239,-0.226931,-0.025675,-0.031743,-0.589388,...,-0.255606,0.062137,0.301709,-0.472495,0.328928,0.529615,0.523701,0.507561,-0.355751,0.259591
7990,149417,-0.328505,0.237160,-0.451192,-0.436901,-0.263571,-0.274049,0.045620,-0.031526,-0.682977,...,-0.263842,0.062594,0.341828,-0.528521,0.320702,0.614759,0.529854,0.605678,-0.406739,0.227682
7991,149452,-0.422952,0.006323,-0.414585,-0.525562,-0.238775,-0.253015,-0.026292,-0.042061,-0.807989,...,-0.175061,-0.058635,0.420576,-0.388243,0.348638,0.737439,0.446040,0.698253,-0.510255,0.070162
7992,149488,-0.291082,0.245929,-0.159964,-0.485578,-0.305785,-0.369605,0.030002,0.197271,-0.558694,...,-0.292513,0.119207,0.125497,-0.263927,0.387574,0.447500,0.883246,0.534130,-0.111221,0.529637


In [29]:
embeddings.to_csv('../data/csv/resnet_triplet_embeddings.csv', index=False)